In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import random

In [2]:
USER_AGENT_LIST = ['Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36']
DRIVER_FILE_PATH = "/Users/qunishdash/Documents/chromedriver_mac64/chromedriver"

In [3]:
def get_chrome_driver(headless_flag):
    # Set up the Selenium webdriver
    from selenium import webdriver
    from selenium.webdriver.chrome.options import Options
    from selenium.webdriver.chrome.service import Service

    service = Service(executable_path=DRIVER_FILE_PATH)
    chrome_options = Options()
    
    if headless_flag:
        # in case you want headless browser  
        chrome_options.add_argument("--headless")
        chrome_options.add_argument("--disable-extensions")
        chrome_options.add_argument("--disable-gpu")
        chrome_options.add_argument("--no-sandbox")
        chrome_options.add_argument("--start-maximized")
        chrome_options.add_argument("user-agent={}".format(random.choice(USER_AGENT_LIST)))
    
    driver = webdriver.Chrome(service=service, options=chrome_options) 

    return driver

In [4]:
driver = get_chrome_driver(False)

In [5]:
import pymongo

client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["getyourguide_data_db"]
collection = db["london"]

In [6]:
from selenium.webdriver.common.by import By
def get_product_data(url):

    driver.get(url)
    try:
        activity_overview = driver.find_element(By.XPATH, '//*[@id="activity-overview"]/p').text.replace("\n", " ")
    except Exception as e:
        activity_overview = ''
        print(e)
    try:
        activity_details = driver.find_element(By.XPATH, '//*[@id="key-details"]/section/dl').text.replace("\n", " ")
    except Exception as e:
        activity_details = ''
    try:
        activity_experience = driver.find_element(By.XPATH, '//*[@id="activity-experience"]').text.replace("\n", " ")
    except Exception as e:
        activity_experience = ''
    try:
        activity_meeting_point = driver.find_element(By.XPATH, '//*[@id="preparation-info"]/section[1]/div/div[2]/p').text.replace("\n", " ")
    except Exception as e:
        activity_meeting_point = ''
    try:
        activity_important_information = driver.find_element(By.XPATH, '//*[@id="section--important-information"]/div/div[2]/div/section').text.replace("\n", " ")
    except Exception as e:
        activity_important_information = ''
    try:
        activity_review_summary = driver.find_element(By.XPATH, '//*[@id="customer-reviews"]/section/div/div[2]/ul').text.replace("\n", " ")
    except Exception as e:
        activity_review_summary = ''

    try:
        activity_provider = driver.find_element(By.XPATH, '//*[@id="main-content"]/section/section[1]/section[4]/div/div/section/div/a').text.replace("\n", " ")
    except Exception as e:
        activity_provider = ''


    # Create a dictionary with the scraped data
    product_data = {
        "activity_url": url,
        "activity_overview": activity_overview,
        "activity_details": activity_details,
        "activity_experience": activity_experience,
        "activity_meeting_point": activity_meeting_point,
        "activity_important_information": activity_important_information,
        "activity_review_summary": activity_review_summary,
        "activity_provider": activity_provider
    }
    
    return product_data

In [7]:
import pandas as pd

if __name__ == "__main__":
    
    # Fetch product URLs from MongoDB
    result = collection.find({}, {"onClickLink": 1,'_id':0})
    result = list(result)
    pv_urls = ["https://www.getyourguide.com" + item['onClickLink']['link'] for item in result]
    
    # Define a list to store scraped data
    scraped_data_list = []

    # Scrape data for each URL and store it in the 'scraped_data_list'
    for url in pv_urls:
        data = get_product_data(url)
        data["activity_url"] = url  # Add the 'activity_url' key to the data dictionary
        scraped_data_list.append(data)

    # Upsert the data into MongoDB based on the 'onClickLink' URLs:
    try:
        for data in scraped_data_list:
            pv_collection = "london_pv"
            pvcollection = db[pv_collection]
            url = data["activity_url"]  # Fetch the URL from the correct location in the data dictionary
            del data["activity_url"]  # Remove the 'activity_url' key as it's not part of the update
            pvcollection.update_one({"activity_url": url}, {"$set": data}, upsert=True)
        print("Data upserted into MongoDB successfully.")
    except Exception as e:
        print("Error while upserting data into MongoDB:", e)

Data upserted into MongoDB successfully.


In [8]:
# Close the MongoDB connection
client.close()

In [10]:
print(scraped_data_list[:2])

[{'activity_overview': "Take a high-speed adventure through the heart of London and then enjoy a guided tour past some great sights like the Tower of London, Tower Bridge, and St. Paul's Cathedral.", 'activity_details': 'Free cancellation Cancel up to 24 hours in advance for a full refund Reserve now & pay later Keep your travel plans flexible — book your spot and pay nothing today. Covid-19 precautions Special health and safety measures are in place. Check your activity voucher once you book for full details. Duration 45 minutes Check availability to see starting times. Live tour guide English', 'activity_experience': "Experience Highlights Blast through London's historic sights fthen a cruise with tour guide Cruise under the iconic Tower Bridge for the selfie of all selfies! Enjoy your friendly crew showing you London at its best Marvel at sights like the Cutty Sark, St. Paul's Cathedral, or Tower of London Full description Meet your friendly skipper and crew at Tower Bridge Quay and

In [11]:
# Close the WebDriver after processing all URLs
driver.quit()